In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<h1> 실버타운 데이터 입지조건 분석 프로젝트 </h1>

- 데이터 : 네이버 지도 '실버타운' 검색 결과 크롤링하기
- 지오서비스웹 사용 : https://www.geoservice.co.kr

In [2]:
# Selenium 설치
!pip install selenium

# chromedriver_autoinstaller 설치
!pip install chromedriver_autoinstaller

# BeautifulSoup 설치
!pip install beautifulsoup4

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,085 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,858 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/u

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import random
from chromedriver_autoinstaller import install as install_chromedriver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import re

import subprocess
import os
import time
import pandas as pd

In [4]:
# Chrome 및 ChromeDriver 설치
def install_chrome():
    subprocess.run(['apt-get', 'update'])
    subprocess.run(['apt-get', 'install', '-y', 'chromium-browser', 'chromium-chromedriver'])
    os.environ['PATH'] += ':/usr/lib/chromium-browser/:/usr/lib/chromium-browser/chromedriver:'

def install_chromedriver():
    subprocess.run(['pip', 'install', 'chromedriver-autoinstaller'])
    import chromedriver_autoinstaller
    chromedriver_autoinstaller.install()

In [5]:

def get_geo_data(driver):
    URL=f"https://pcmap.place.naver.com/place/list?query=%EC%8B%A4%EB%B2%84%ED%83%80%EC%9A%B4&x=127.482783&y=34.963967&clientX=127.482783&clientY=34.963967&display=70&ts=1717549151804&mapUrl=https%3A%2F%2Fmap.naver.com%2Fp%2Fsearch%2F%EC%8B%A4%EB%B2%84%ED%83%80%EC%9A%B4"
    driver.get(URL)
    time.sleep(5)

    data_list = []
    page_list = count_pages(driver)

    for page in page_list :
      page.click()
      print(page)

      #페이지 스크롤
      scroll_page(driver)

      #li 요소별로 크롤링
      item_list = driver.find_elements(By.CSS_SELECTOR,'li.VLTHu')
      for item in item_list:
        try:
          name = item.find_element(By.CSS_SELECTOR, '.YwYLL').text
          type = item.find_element(By.CSS_SELECTOR, '.YzBgS').text

          road_name = ""
          parcel_number = ""
          postal_code = ""

          # 주소 버튼 클릭
          address_button = item.find_element(By.CSS_SELECTOR, 'a.uFxr1')
          address_button.click()
          time.sleep(2)

          addr = driver.find_elements(By.CSS_SELECTOR, 'div.zZfO1')
          pattern = re.compile(r'(도로명|지번|우|우편번호|편번호|복사)')
          place_part = '\n'.join([element.text for element in addr])
          place_texts = place_part.replace('\n','').split('복사')

          for place in place_texts:
            if '도로명' in place: road_name = pattern.sub('', place_texts[0])
            if '지번' in place: parcel_number = pattern.sub('', place_texts[1])
            if '우' in place: postal_code = pattern.sub('', place_texts[2])

          town_data = {
            'Name': name,
            'Type': type,
            'RoadName': road_name,
            'ParcelNumber': parcel_number,
            'PostalCode' : postal_code
          }
          data_list.append(town_data)
          print(town_data)

        except NoSuchElementException:
            print("필요한 요소를 찾을 수 없습니다.")
            continue

    return data_list


In [6]:
def count_pages(driver):
  page_list = driver.find_elements(By.CSS_SELECTOR,'a.mBN2s')
  return page_list

In [7]:
def scroll_page(driver):
    index = 0
    while True:
        driver.execute_script("document.querySelector('.Ryr1F').scrollTop += 1000;")
        time.sleep(2)
        index += 1

        if(index == 10) : break

In [13]:
def main():

    # Chrome 및 ChromeDriver 설치
    install_chrome()
    install_chromedriver()

    # Chrome 옵션 설정
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Headless 모드로 실행
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=chrome_options)

    try:
      list = get_geo_data(driver)

      df = pd.DataFrame(list, columns=['Name', 'Type', 'RoadName', 'ParcelNumber', 'PostalCode'])
      return df
    finally:
        # 드라이버 종료
        driver.quit()

if __name__ == '__main__':
  final_df = main()
  final_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Github/Data_Analysis/SilverTown/SilverTownData.csv', index=False)

<selenium.webdriver.remote.webelement.WebElement (session="ad5eb09c0e7246644cddf23e998681c3", element="f.75DD1F773C043522E160DD4A8C8702A9.d.024BA56923743A6A3FA5C873CF95C879.e.5")>
{'Name': '청담실버타운5차', 'Type': '주택', 'RoadName': '서울 강남구 도산대로72길 31', 'ParcelNumber': '청담동 13-41', 'PostalCode': '06063'}
{'Name': '신우실버타운', 'Type': '주택', 'RoadName': '서울 강남구 학동로82길 28', 'ParcelNumber': '삼성동 65-6', 'PostalCode': '06084'}
{'Name': '실버타운', 'Type': '스파게티,파스타전문', 'RoadName': '서울 관악구 남부순환로214길 30 1층 실버타운', 'ParcelNumber': '봉천동 896-24 1층 실버타운', 'PostalCode': '08786'}
{'Name': '치악산실버타운', 'Type': '노인복지', 'RoadName': '강원 횡성군 강림면 태종로 92', 'ParcelNumber': '강림리 744-5', 'PostalCode': '25272'}
{'Name': '호세실버타운', 'Type': '노인복지', 'RoadName': '인천 강화군 불은면 고려왕릉로 376 호세요양원', 'ParcelNumber': '두운리 342-5', 'PostalCode': '23046'}
{'Name': '실버타운', 'Type': '노인복지', 'RoadName': '경기 이천시 영창로 188', 'ParcelNumber': '창전동 158-1', 'PostalCode': '17369'}
{'Name': '실버타운', 'Type': '주택', 'RoadName': '경북 경산시 자인면 북사로7길 30', 'ParcelNum

In [14]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Github/Data_Analysis/SilverTown/SilverTownData.csv')
df

,Name,Type,RoadName,ParcelNumber,PostalCode
0,청담실버타운5차,주택,서울 강남구 도산대로72길 31,청담동 13-41,6063.0
1,신우실버타운,주택,서울 강남구 학동로82길 28,삼성동 65-6,6084.0
2,실버타운,"스파게티,파스타전문",서울 관악구 남부순환로214길 30 1층 실버타운,봉천동 896-24 1층 실버타운,8786.0
3,치악산실버타운,노인복지,강원 횡성군 강림면 태종로 92,강림리 744-5,25272.0
4,호세실버타운,노인복지,인천 강화군 불은면 고려왕릉로 376 호세요양원,두운리 342-5,23046.0
...,...,...,...,...,...
274,계양데이케어센터,노인복지,인천 계양구 아나지로 191,효성동 273-101,21112.0
275,민들레실버요양원,"요양원,요양센터",울산 울주군 온양읍 솔밭1길 30-9 민들레실버요양원,대안리 204-2 민들레실버요양원,NaN
276,푸른마을,노인복지,인천 강화군 강화읍 갑룡길206번길 55 푸른마을,용정리 865-2,23023.0
277,백세만세실버케어,"요양원,요양센터","경기 남양주시 진접읍 봉현로 67-5 1,2동","장현리 570 1,2동",12007.0


<h3> 지오웹서비스 사용하여 주소 데이터를 좌표로 변환 </h3>
→ SilverTownGeoData.csv

In [16]:
geo_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Github/Data_Analysis/SilverTown/SilverTownGeoData.csv')
geo_df

,field1,field2,field3,field4,field5,_GC_TYPE,_CLEANADDR,_X,_Y
0,청담실버타운5차,주택,서울 강남구 도산대로72길 31,청담동 13-41,6063.0,정,서울특별시 강남구 청담동 13-41,127.045851,37.522260
1,신우실버타운,주택,서울 강남구 학동로82길 28,삼성동 65-6,6084.0,정,서울특별시 강남구 삼성동 65-6,127.051894,37.517074
2,호세실버타운,노인복지,인천 강화군 불은면 고려왕릉로 376 호세요양원,두운리 342-5,23046.0,정,인천광역시 강화군 불은면 두운리 342-5,126.477926,37.675357
3,실버타운,"스파게티,파스타전문",서울 관악구 남부순환로214길 30 1층 실버타운,봉천동 896-24 1층 실버타운,8786.0,정,서울특별시 관악구 봉천동 896-24,126.947395,37.480304
4,치악산실버타운,노인복지,강원 횡성군 강림면 태종로 92,강림리 744-5,25272.0,정,강원특별자치도 횡성군 강림면 강림리 744-5,128.120994,37.357565
...,...,...,...,...,...,...,...,...,...
265,주안 에쉐르카운티,노인복지,인천 미추홀구 주승로 110,주안동 1216-17,22231.0,정,인천광역시 미추홀구 주안동 1216-17,126.682657,37.445068
266,아산실버요양원,"요양원,요양센터",충남 아산시 송악면 외암로 759 아산실버요양원,마곡리 165-1 아산실버요양원,31583.0,정,경상북도 경산시 진량읍 마곡리 165-1,128.846678,35.846351
267,민들레실버요양원,"요양원,요양센터",울산 울주군 온양읍 솔밭1길 30-9 민들레실버요양원,대안리 204-2 민들레실버요양원,NaN,정,경상북도 영천시 화산면 대안리 204-2,128.827800,36.030962
268,백세만세실버케어,"요양원,요양센터","경기 남양주시 진접읍 봉현로 67-5 1,2동","장현리 570 1,2동",12007.0,정,충청남도 보령시 청라면 장현리 570-1,126.673640,36.422286
